# Exercise 4
## Lillian Mueller and Regina Hong 
Comparing models using cross validation

In [61]:
from sklearn import (
    linear_model, 
    preprocessing,
    model_selection,
    metrics, 
    tree)
from sklearn.datasets import load_iris
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [67]:
# loading in the iris dataset from sklearn
iris_data = load_iris()

# turning dataset into dataframe format for easier reading
df_iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)

# adding the target as a class column and renaming numbers to class names
# 0 = setosa, 1 = versicolor, 2 = virginica

df_iris['class'] = iris_data.target.tolist()
df_iris['class'].replace({0: 'setosa', 1: 'versicolor', 2: 'virginica'},
                inplace=True)

#Create a new column for the target (to be ready for processing)
label_obj = preprocessing.LabelEncoder()
df_iris['class_level'] = label_obj.fit_transform(df_iris['class'])

print("Full Dataset:")
df_iris.sample(4)

Full Dataset:


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class,class_level
47,4.6,3.2,1.4,0.2,setosa,0
43,5.0,3.5,1.6,0.6,setosa,0
64,5.6,2.9,3.6,1.3,versicolor,1
115,6.4,3.2,5.3,2.3,virginica,2


In [72]:
# cross validation of logistic model without penality
# using without penality highest accuracy as shown in report 3
predictors, target = iris_data.data, iris_data.target

logR_cross_val = model_selection.cross_val_score(
    linear_model.LogisticRegression(penalty=None), 
    predictors, 
    target, 
    cv = None,  # default -> 5-fold cross validation
    scoring='accuracy'
)
print("Cross Validation of Logistic Regression without Penalty")
logR = pd.DataFrame(logR_cross_val)
print(logR)

Cross Validation of Logistic Regression without Penalty
          0
0  1.000000
1  1.000000
2  0.933333
3  0.933333
4  1.000000


In [73]:
# cross validation of decision tree with entropy
# using entropy method showed highest accuracy as shown in report 2
dtree_cross_val = model_selection.cross_val_score(
    tree.DecisionTreeClassifier(criterion='entropy'), 
    predictors, 
    target, 
    cv = None,  # default -> 5-fold cross validation
    scoring='accuracy'
)
print("Cross Validation using Decision Tree with Entropy")
dtree = pd.DataFrame(dtree_cross_val)
print(dtree)

Cross Validation using Decision Tree with Entropy
          0
0  0.966667
1  0.966667
2  0.900000
3  0.933333
4  1.000000


In [90]:
# Comparison of score means and standard deviation 
# create table comparing average and std of multiple dataframes
def comparison_table(dfs, columns):
    meanScores = pd.concat(
        [df.mean() for df in dfs], 
        axis=1)
    meanScores.columns = [col for col in columns]

    stdScores = pd.concat(
        [df.std() for df in dfs], 
        axis=1)
    stdScores.columns = [col for col in columns]

    comparison = pd.concat([meanScores, stdScores], ignore_index=True)
    comparison.index = ['Mean', 'Stardard Dev.']
    return comparison

comparison_table([logR, dtree], ['Logistic Regression', 'Decision Tree'])

,Logistic Regression,Decision Tree
Mean,0.973333,0.953333
Stardard Dev.,0.036515,0.038006


In [91]:
# create function to perform manual cross validation
def perform_manual_cross_validation(dataset, feature_cols, target_col, model, numFolds):
    accuracy = [] # collect accuracy of each fold
    fulldataset = dataset.copy() # keep copy of full dataset

    for i in range(numFolds):
        # get separate group for traning fold
        # since dataframe is getting smaller, must adjust the fraction of entries taken each time
        train_fold = dataset.sample(frac=(1/(numFolds-i)))

        # get remaining dataset to use a test folds
        test_folds = fulldataset.drop(train_fold.index)

        # fit model 
        mod = model.fit(train_fold[feature_cols], train_fold[target_col])
        
        # get accuracy of model
        accuracy.append(
            metrics.accuracy_score(
                test_folds[target_col], 
                mod.predict(test_folds[feature_cols]))
        )

        # keep track of unused data entries 
        dataset.drop(train_fold.index, inplace=True)
    
    return accuracy


In [92]:
print("Cross Validation of Logistic Regression without Penalty")
logR_accuracy = perform_manual_cross_validation(
    df_iris.copy(), 
    ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'], 
    'class_level', 
    linear_model.LogisticRegression(penalty=None), 
    5
    )
accuracy = pd.DataFrame()
accuracy['Logistic Regression'] = logR_accuracy
accuracy['Logistic Regression']

Cross Validation of Logistic Regression without Penalty


0    0.958333
1    0.958333
2    0.841667
3    0.891667
4    0.966667
Name: Logistic Regression, dtype: float64

In [93]:
print("Cross Validation using Decision Tree with Entropy")
dtree_accuracy = perform_manual_cross_validation(
    df_iris.copy(), 
    ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'], 
    'class_level', 
    tree.DecisionTreeClassifier(criterion='entropy'), 
    5
    )
accuracy['Decision Tree'] = dtree_accuracy
accuracy['Decision Tree']

Cross Validation using Decision Tree with Entropy


0    0.933333
1    0.941667
2    0.950000
3    0.950000
4    0.950000
Name: Decision Tree, dtype: float64

In [83]:
print('Average')
print(accuracy.mean())
print('Standard Deviation')
print(accuracy.std())

Average
Logistic Regression    0.956667
Decision Tree          0.898333
dtype: float64
Standard Deviation
Logistic Regression    0.010865
Decision Tree          0.072265
dtype: float64
